In [ ]:
#dependencies
import h5py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import utils.plotutils as plu
import utils.imreadin as imr

%matplotlib inline

In [ ]:
# Model Parameters
nneurons = 100
patchsize = 128
batchsize = 100
iterations = 100000
testiterations = 10000

#noise - these Prateek is interested in changing to removed iid asumption
noisexsigma = 0.8
noisersigma = 2

#to keep covariance matrix small for now
npatchestotal =  iterations*batchsize #np.int(1e6)

lambdaj = 0.001 #this is adjusted in order to get ravg = 1
ravg = 1
#minact = 1e-1
#bslfr = 1e-1

learning_rate = 1

In [ ]:
#full images
#vhims = extract_images('../vanHaterenNaturalImages/VanHaterenNaturalImagesCurated.h5')

#image patches (as in Karklin& Simoncelli)
#only load in image patches if we haven't yet.
try:
    vhims
except NameError:
    print("Loading Van Hateren Natural Image Database...")
    vhims = imr.vanHateren(
        img_dir='../vanHaterenNaturalImages/VanHaterenNaturalImagesCurated.h5',
        normalize = True,
        patch_edge_size=patchsize
        )
    print("Done Loading!")    
    np.random.shuffle(vhims.images)
    print("Done Shuffling!")
    
    ##reduce images so we only hold in memory the ones we're using
    vhims.images = vhims.images[0:npatchestotal]
    
print("Images Loaded. Carry On!")

#code to save if i had enough hd space. i shold buy a new computer.
#f=open('../vanHaterenNaturalImages/vhnormpatches_{}.npy'.format(patchsize),'wb')
#np.save(f,vhims)
#f.close()

In [ ]:
#example image
plt.imshow(vhims.images[400,:,:],cmap='gray', interpolation='None')

In [ ]:
import os
import numpy as np
import h5py

filename ='../vanHaterenNaturalImages/VanHaterenNaturalImagesCurated.h5'
normalize = True,
patch_edge_size=128
normalize=False

with h5py.File(filename, "r") as f:
    full_img_data = np.array(f['van_hateren_good'], dtype=np.float32)
    if(normalize):
        print('normalizing...')
        full_img_data = full_img_data - np.mean(full_img_data,axis=(1,2),keepdims=True)
        full_img_data = full_img_data/np.std(full_img_data,axis=(1,2),keepdims=True)
    if patch_edge_size is not None:
        print('sectioning into patches....')
        print(full_img_data.shape)
        (num_img, num_px_rows, num_px_cols) = full_img_data.shape
        num_img_px = num_px_rows * num_px_cols
        #print(num_px_rows)
        #print(patch_edge_size)
        #print(num_px_rows % patch_edge_size)
        #make sure tiling is compatible with dimensions of image
        assert (num_px_rows % patch_edge_size == 0)  , ("The number of image row edge pixels % the patch edge size must be 0.")
        assert (num_px_cols % patch_edge_size == 0)  , ("The number of image column edge pixels % the patch edge size must be 0.")
        num_patches = int(num_img_px / patch_edge_size**2)
        #full_img_data = np.reshape(full_img_data, (num_img, num_img_px)
        #data = np.vstack([full_img_data[idx,...].reshape(num_patches, patch_edge_size, patch_edge_size) for idx in range(num_img)])
        
        
    else:
        data = full_img_data
        num_patches = 0

In [ ]:
data = np.asarray(np.split(full_img_data, num_px_cols/patch_edge_size,2)) # tile column-wise
data = np.asarray(np.split(data, num_px_rows/patch_edge_size,2)) #tile row-wise
data = np.reshape(np.transpose(data,(3,4,0,1,2)),(patch_edge_size,patch_edge_size,-1)) #stack tiles together

In [ ]:
np.shape(test)
plt.imshow(test[:,:,1000],cmap='gray', interpolation='None')


In [ ]:
#covariance parameters

#global curvature of images
try:
    cx
except NameError:
    print("Couldn't find C_x matrix. Calculating now....")
    xr = vhims.images[0:npatchestotal,:,:].reshape(-1,patchsize**2).T.astype('float32')
    cx = np.cov(xr)
    cxnorm = cx-np.mean(cx) # a normalized version
print("C_x calculated. Shape is {}, calculated from {} patches.".format(np.shape(cx),npatchestotal))

#cx = np.cov(xr)
#cx = np.dot(xr,xr.T)/nimagestest - np.dot(np.mean(xr,axis=1,keepdims=True).T,np.mean(xr,axis=1,keepdims=True))
cxnorm= np.eye(patchsize**2)
#print(cx)
#print(np.linalg.det(cx))
#plt.pcolormesh(np.linalg.inv(cx))
plt.pcolormesh(cxnorm)
plt.colorbar()

#noise covariances
cnx = np.eye(patchsize**2) * noisexsigma**2
cnr = np.eye(nneurons) * noisersigma**2

In [ ]:
#input image
x = tf.placeholder(tf.float32, shape=(patchsize**2,batchsize),name='input_image')

#noise
nx = tf.placeholder(tf.float32, shape=(patchsize**2,batchsize),name='input_noise')
nr = tf.placeholder(tf.float32, shape=(nneurons,batchsize),name='response_noise')

#covariances
cxt = tf.constant(cxnorm,dtype=tf.float32)#+0.01
cnxt = tf.constant(cnx,dtype=tf.float32)#+0.01
cnrt = tf.constant(cnr,dtype=tf.float32)#+0.01

#weights
w = tf.Variable(tf.random_normal(shape=(patchsize**2,nneurons),dtype=tf.float32,stddev=1),name='weights')

#relu params - both positive
actslope = tf.Variable(np.abs(tf.random_normal([nneurons,1],stddev=1,dtype=tf.float32)),dtype=tf.float32,name='relu_slope')
actoffset = tf.Variable(np.abs(tf.random_normal([nneurons,1],stddev=1,dtype=tf.float32)),dtype=tf.float32,name='relu_offset')

#activation function
def af(slope,offset,y):
    #return(1/(1+tf.exp(-1*slope*(y+offset)))) #sigmoid
    return(tf.mul(tf.abs(slope),tf.nn.relu(tf.add(y,offset)))) #relu

def af_ddy(slope,offset,y):
    #return(slope*af(slope,offset,y)*(1-af(slope,offset,y))) #sigmoid
    return(tf.mul(tf.cast(tf.less(offset, y),tf.float32),tf.abs(slope))) #relu
    
#calculate response
y = tf.transpose(tf.matmul(tf.transpose(x + nx),w))
r = tf.add(af(actslope,actoffset,y), nr)
rate = tf.reduce_mean(r,1)
meanrate = tf.reduce_mean(rate)

#objective function: maximize I(X,R)-sum_j(lambda*<r_j>)
#1s for all places respose (y + offset) is greater than zero
#mask = tf.cast(tf.less(tf.zeros((nneurons,1)),y+reluoff),tf.float32)
#multiply response by reluslope everywhere response was greater than zero, add baseline to this.
g = tf.diag(tf.reshape(tf.reduce_mean(af_ddy(actslope,actoffset,y),axis=1),[nneurons]))

cxr = tf.matrix_inverse(tf.matrix_inverse(cxt) + #with inverse calculated by Shariq
#cxr =  1 * (tf.matrix_inverse(cxt) + #as in paper
                        tf.matmul(w, 
                        tf.matmul(g, 
                        tf.matmul(tf.matrix_inverse(
                        tf.matmul(g,
                        tf.matmul(tf.transpose(w), 
                        tf.matmul(cnxt, 
                        tf.matmul(w, g)))) + cnrt),
                        tf.matmul(g, tf.transpose(w))))))


cxr = tf.div(cxr,tf.reduce_max(cxr)) #scale to make determinant smaller

#informion = tf.reduce_mean(0.5*tf.log1p(2*np.pi*np.exp(1)*(5)))
det_g = tf.matrix_determinant(g)
detcxrt = tf.matrix_determinant(cxr)
information = 0.5*tf.log1p(2*np.pi*np.exp(1)*(tf.matrix_determinant(cxr)))
#information = 0.5*tf.log1p(2*np.pi*np.exp(1)*tf.trace(cxr))
objective =  information - lambdaj * tf.reduce_sum(rate)#/nneurons

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(-objective)

#run it!
#sess = tf.Session
with tf.Session() as sess:
    #initialize vars
    init = tf.global_variables_initializer()
    sess.run(init)
    
    #save evolution of system over training
    information_evolution = []
    objective_evolution = []
    rate_evolution = []
    detcxrs_evolution = []
    
    hist = {}
    
    #train over niterations
    nits = iterations #full run
    nits = testiterations #test run
    
    print('Training {} iterations...'.format(nits))
    for ii in range(nits):
        if(ii%(int(nits/10))==0):
            print(str(ii)+', ',end="")
        #image = np.random.rand(patchsize**2,batchsize).astype(np.float32) #random image
        image = np.reshape(vhims.images[ii*batchsize:(1+ii)*batchsize,:,:],(batchsize,patchsize**2)).T.astype(np.float32)
        noise_input = np.random.normal(0,noisexsigma,[patchsize**2,batchsize]).astype(np.float32)
        #noise_input = np.zeros([patchsize**2,1]).astype(np.float32)
        noise_response = np.random.normal(0,noisersigma,[nneurons,batchsize]).astype(np.float32)
        #noise_response = np.zeros([nneurons,1]).astype(np.float32)
        hist[0] = image
        hist[1] = noise_input
        hist[2] = noise_response
        
        #train
        #train_step.run(feed_dict={x:image, nx: noise_input, nr:noise_response})
        sess.run(train_step, feed_dict={x:image, nx: noise_input, nr:noise_response})
        
        #save evolution of params
        info = sess.run(information, feed_dict={x:image, nx: noise_input, nr:noise_response})
        information_evolution.append(info)
        cost = sess.run(objective, feed_dict={x:image, nx: noise_input, nr:noise_response})
        objective_evolution.append(cost)
        rate = sess.run(meanrate, feed_dict={x:image, nx: noise_input, nr:noise_response})
        rate_evolution.append(rate)
        detcxr = sess.run(detcxrt, feed_dict={x:image, nx: noise_input, nr:noise_response})
        detcxrs_evolution.append(detcxr)
#       if ii%iterations == 0:
#            pass
        
    #print statements
    #myweights = w.eval()
    #print(myweights)
    #myrlus = reluslope.eval()
    #print(myrlus)
    #myrluof = reluoff.e`val()
    #print(myrluof)
    #myrs = r.eval()
    #print(myrs)
    #myobjective = objective.eval()
    #print(myobjective)
    weights = sess.run(w, feed_dict={x:image, nx: noise_input, nr:noise_response})
    detz = sess.run(g, feed_dict={x:image, nx: noise_input, nr:noise_response})
    print('Done!')
    

In [ ]:
plt.figure(figsize=(6,10))
plt.subplot(2,2,1,title='Information')
plt.plot(information_evolution)

plt.subplot(2,2,2,title='Firing Rate')
plt.plot(rate_evolution)

plt.subplot(2,2,3,title='Objective')
plt.plot(objective_evolution)

plt.subplot(2,2,4,title='cxrs Determinant')
plt.plot(detcxrs_evolution)

In [ ]:
wr = np.rollaxis(np.reshape(weights,(patchsize,patchsize,nneurons)),2)
plu.display_data_tiled(wr, normalize=False, title="weights", prev_fig=None)

In [ ]:
plt.imshow(np.reshape(weights[:,6],(patchsize,patchsize)),interpolation='none')

## Bonus Code

In [ ]:
### weights
w = np.random.randn(patchsize**2,nneurons)
#relu params - both positive
actslope = np.abs(np.random.randn(nneurons,1))
actoffset = np.abs(np.random.randn(nneurons,1))


def sigmoid_af(slope,offset,y):
    #return(1/(1+tf.exp(-1*slope*(y+offset))))
    return(np.abs(slope)*((y-offset)>0))

def sigmoid_af_ddy(slope,offset,y):
    #return(slope*sigmoid_af(slope,offset,y)*(1-sigmoid_af(slope,offset,y)))
    mask = (offset[:,0] < y.T)
    return(mask*np.abs(slope[:,0]))



#calculate response
y = np.transpose(np.dot(np.transpose(hist[0][:,0] + hist[1][:,0]),w))
#print(np.transpose(hist[0] + hist[1]).shape)
#print(w.shape)
#print(y.shape)
r = np.add(sigmoid_af(actslope, actoffset, y), hist[2][:,0])
rate = np.mean(r,0)
#print(rate)

#g = np.diag(np.reshape(sigmoid_af_ddy(actslope,actoffset,y),-1)+0.1) 
g = np.diag(np.reshape(sigmoid_af_ddy(actslope,actoffset,y),[nneurons]))

print('g')
print(g)
print(np.linalg.det(g))
print(np.trace(g))

print('cxnorm')
print(cxnorm)
print(np.linalg.det(cxnorm))

print('cnx')
print(cnx)
print(np.linalg.det(cnx))

print('cnr')
print(cnr)
print(np.linalg.det(cnr))


cxr = np.linalg.inv(np.linalg.inv(cx) + #with inverse calculated by Shariq
#cxr =  1 * (tf.matrix_inverse(cx) + #as in paper
                        np.dot(w, 
                        np.dot(g, 
                        np.dot(np.linalg.inv(
                        np.dot(g,
                        np.dot(np.transpose(w), 
                        np.dot(cnx, 
                        np.dot(w, g)))) + cnr),
                        np.dot(g, np.transpose(w))))))

print('cxr')
print(np.linalg.det(cxr))
print(cxr)

print('gwcwg')
pgwcwg = np.linalg.inv(np.dot(g,np.dot(w.T,np.dot(cnx,np.dot(w,g))))+cnr)
gwcwg = np.dot(w,np.dot(g,np.dot(pgwcwg,np.dot(g,w.T))))+np.linalg.inv(cx)

print(gwcwg)

print(np.linalg.det(gwcwg))

plt.pcolormesh(gwcwg)
plt.colorbar()

information = np.mean(0.5*np.log1p(2*np.pi*np.exp(1)*(np.linalg.det(cxr))))
objective =  information - np.sum(lambdaj * rate)#/nneurons

print(information)
#train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(-objective)

In [ ]:
np.mean(cx-np.mean(cx))